In [6]:
import itertools
from tree_from_assoc_rules import list_all_factors
from controller import generate_decision_tree
from tabulate import tabulate
import datetime

In [7]:
# Define all factors - Ref(3)
# all_factors = ["stay", "length_of_stay", "country", "review_group", "room_type"]
all_factors = ["stay", "length_of_stay", "review_group"]

# Read file of possible values
possible_factor_values = list_all_factors()

In [8]:
# All possible permutations of all factors - Ref(4)
all_factors_combinations = []
for length in range(3, 4): # เพื่อทดลอง เลยลดจำนวนลง ของเดิม = len(all_factors) + 1
    all_factors_combinations += list(itertools.combinations(all_factors, length))

print(all_factors_combinations)

[('stay', 'length_of_stay', 'review_group')]


In [9]:
# Hardcoded some constants
mcda_type = "rating"
hotel_name = "Amari Phuket"
rule_year = None
rule_month = None

weights_experiment_1 = [1/3, 1/3, 1/3]
weights_experiment_2 = [1/8, 3/8, 4/8]

In [10]:
print("all cases = " + str(len(all_factors_combinations)))
def collect_paths(node, route_nodes=[]):
    result = []
    has_child = len(node.children) > 0
    if has_child:
        for child_node in node.children:
            new_route_nodes = []
            new_route_nodes += route_nodes
            new_route_nodes.append(node)
            result += collect_paths(child_node, route_nodes=new_route_nodes)
    else:
        new_route_nodes = []
        new_route_nodes += route_nodes
        new_route_nodes.append(node)
        return [new_route_nodes]
    return result

# Generate decision tree for all permutations
count = 0
gen_count = 0
nodes = []
result_score_table = [["Outcome", "Factor_1", "Factor_2", "Factor_3", "Factor_4", "Factor_5", "Score"]]
for idx, permutation in enumerate(all_factors_combinations):
    # Calculate weights array
    factors_length = len(permutation)
    factor_weight = 1 / factors_length
    weights = weights_experiment_2

    # Generate Decision Tree
    # print(permutation)
    gen_count += 1
    a = datetime.datetime.now()
    root_node = generate_decision_tree(mcda_type, list(permutation), weights, hotel_name=hotel_name,
                                       rule_year=rule_year, rule_month=rule_month)
    b = datetime.datetime.now()
    c = b - a
    print(str(gen_count) + ": " + str(c) + " - " + str(permutation))
    nodes.append(root_node)

    # Prototype: calculate score on each tree
    # print(RenderTree(root_node, style=AsciiStyle()).by_attr())
    paths = collect_paths(root_node)
    for p_idx, path in enumerate(paths):
        count += 1
        result_name = list(map(lambda k : k.name, path[1:]))
        weight_sum = sum(n.weight for n in path[1:])
        score_sum = sum(n.score for n in path[1:]) if weight_sum == 1 else 0
        result_row = [None] * 7
        result_row[0] = count
        result_row[1:5] = result_name + [None] * (5 - len(result_name))
        result_row[6] = score_sum
        result_score_table.append(result_row)

all cases = 1
1: 0:00:05.856357 - ('stay', 'length_of_stay', 'review_group')


In [11]:
import csv

# print(tabulate(result_score_table, headers='firstrow', tablefmt='fancy_grid'))
with open("experiment-2-neq.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(result_score_table)